In [1]:
import os

import pandas as pd
import seaborn as sns

In [2]:
data_in = os.path.join('data', 'raw')
data_out = os.path.join('data', 'preprocessed')

day = 3

if day == 1:
    positions_csv = 'positions_day_0_day_63.csv'
    sales_csv = 'stock_and_sales_day_0_day_63.csv'
elif day == 2:
    positions_csv = 'positions_day_0_day_78.csv'
    sales_csv = 'stock_and_sales_day_0_day_77.csv'
elif day == 3:
    positions_csv = 'positions_day_0_day_85.csv'
    sales_csv = 'stock_and_sales_day_0_day_84.csv'

In [3]:
df_pos = pd.read_csv(os.path.join(data_in, positions_csv))
max_date_number_pos = df_pos['date_number'].max()
print(f'Positions 0 - {max_date_number_pos} (shape {df_pos.shape})')
display(df_pos.head())
print('******************************************')

df_sales = pd.read_csv(os.path.join(data_in, sales_csv))
max_date_number_sales = df_pos['date_number'].max()
assert max_date_number_pos == max_date_number_sales, 'Different end dates in positions and sales'
print(f'Stock and sales 0 - {max_date_number_sales} (shape {df_sales.shape})')
display(df_sales.head())
print('******************************************')

df_product_blocks = pd.read_csv(os.path.join(data_in, 'product_blocks.csv'))
print(f'Product block (shape {df_product_blocks.shape})')
display(df_product_blocks.head())
print('******************************************')

df_products = pd.read_csv(os.path.join(data_in, 'products.csv'))
print(f'Products (shape {df_products.shape})')
display(df_products.head())
print('******************************************')

Positions 0 - 85 (shape (1389358, 4))


,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


******************************************
Stock and sales 0 - 85 (shape (3418281, 6))


,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


******************************************
Product block (shape (15238, 2))


,product_id,block_id
0,612967398,0
1,296892108,0
2,139541214,0
3,963923934,0
4,938230141,0


******************************************
Products (shape (15238, 4))


,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


******************************************


In [4]:
nb_products_products_table = df_products.nunique()['product_id']
nb_products_sales_table = df_sales.nunique()['product_id']
nb_days = df_sales.nunique()['date_number']
print(f'nb_products_products_table = {nb_products_products_table}. nb_products_sales_table = {nb_products_sales_table}')
print(f'nb_products ({nb_products_sales_table}) * nb_days ({nb_days}) = {nb_products_products_table * nb_days}')

nb_products_products_table = 15238. nb_products_sales_table = 9541
nb_products (9541) * nb_days (85) = 1295230


In [5]:
df_pos_rank = df_pos.iloc[df_pos.groupby(['date_number', 'product_id'], group_keys=False)['position'].idxmin()]
df_pos_rank = df_pos_rank.rename(columns={'position': 'best_position'})
display(df_pos_rank.head())

,date_number,product_id,category_id,best_position
7893,0,310130,729182307,3
10769,0,1178388,975244833,19
6057,0,1561460,563537811,3
3590,0,1874414,367730119,12
2590,0,2094841,251700555,48


In [6]:
df_sales_prod = df_sales.groupby(['date_number', 'product_id']).sum()
df_sales_prod = df_sales_prod.join(df_pos_rank.set_index(['date_number', 'product_id'])).reset_index()
df_sales_prod = df_sales_prod.join(df_products.set_index('product_id'), on='product_id')
df_sales_prod = df_sales_prod.join(df_product_blocks.set_index('product_id'), on='product_id')
df_sales_prod['revenue'] = df_sales_prod['sales'] * df_sales_prod['price']
df_sales_prod = df_sales_prod[['date_number', 'product_id', 'block_id', 'sales', 'price', 'revenue', 'stock', 'best_position']]
df_sales_prod[['sales', 'price', 'revenue', 'best_position']] = df_sales_prod[['sales', 'price', 'revenue', 'best_position']].apply(pd.to_numeric)
print(f'Sales and stock per product per day (shape {df_sales_prod.shape})')
print('Number of unique series', df_sales_prod.nunique()['product_id'])
df_sales_prod.head()

Sales and stock per product per day (shape (596764, 8))
Number of unique series 9541


,date_number,product_id,block_id,sales,price,revenue,stock,best_position
0,0,310130,1726,11,12.95,142.45,461,3.0
1,0,1178388,592,0,49.95,0.00,60,19.0
2,0,1561460,1625,7,29.95,209.65,791,3.0
3,0,1874414,1135,4,25.95,103.80,281,12.0
4,0,2436420,779,0,25.95,0.00,245,NaN


In [7]:
df_sales_block = df_sales_prod.drop('product_id', axis=1).groupby(['date_number', 'block_id']).sum().reset_index()
df_sales_block = df_sales_block.drop(['price', 'sales', 'stock', 'best_position'], axis=1)
print(f'Sales and stock per block per day (shape {df_sales_block.shape})')
print('Number of unique series', df_sales_block.nunique()['block_id'])
df_sales_block.head()

Sales and stock per block per day (shape (204304, 3))
Number of unique series 2602


,date_number,block_id,revenue
0,0,0,674.60
1,0,1,29.95
2,0,2,679.40
3,0,3,53.91
4,0,4,1267.90


In [8]:
df_sales_prod_dataset = df_sales_prod.pivot_table(index='product_id', columns='date_number', values='revenue')
df_sales_prod_dataset.columns = list(map(lambda date: f'X{date}', df_sales_prod_dataset.columns))
display(df_sales_prod_dataset.head())

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84
product_id,,,,,,,,,,,,,,,,,,,,,
151926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,103.80,155.70,155.70,129.75,285.45,25.95,129.75,103.80,51.90,51.90
213413,NaN,NaN,NaN,59.85,159.60,259.35,518.7,418.95,179.55,279.30,...,159.60,199.50,179.55,299.25,199.50,259.35,139.65,239.40,279.30,159.60
310130,142.45,168.35,181.3,194.25,220.15,233.10,207.2,233.10,233.10,194.25,...,64.75,64.75,129.50,90.65,38.85,12.95,25.90,77.70,77.70,64.75
455200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,59.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
571044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.85,79.75,15.95,63.80,669.90,255.20,303.05,239.25,271.15,143.55


In [9]:
df_sales_block_dataset = df_sales_block.pivot_table(index='block_id', columns='date_number', values='revenue')
df_sales_block_dataset.columns = list(map(lambda date: f'X{date}', df_sales_block_dataset.columns))
display(df_sales_block_dataset.head())

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84
block_id,,,,,,,,,,,,,,,,,,,,,
0,674.60,656.90,403.20,950.40,764.65,481.15,163.65,619.05,419.10,327.45,...,309.50,239.65,359.45,339.45,7290.75,2676.65,1827.65,709.05,389.45,888.85
1,29.95,149.75,89.85,179.70,359.40,59.90,89.85,59.90,149.75,29.95,...,1731.65,1540.00,2034.75,1129.80,1409.15,1176.00,1314.75,1316.60,1105.35,940.45
2,679.40,1228.90,789.25,1138.95,1258.90,1318.85,779.30,689.40,569.50,779.25,...,455.40,1008.85,629.30,669.15,1014.85,569.35,719.25,549.45,359.65,849.15
3,53.91,5.99,41.93,83.86,113.81,101.83,89.85,71.88,83.86,59.90,...,79.90,39.95,45.94,0.00,0.00,39.95,0.00,0.00,39.95,0.00
4,1267.90,1627.20,1008.25,998.20,1647.30,898.40,848.60,1108.10,688.90,808.50,...,327.20,720.65,485.05,1032.15,6455.10,3116.75,2875.20,2775.15,2831.25,2759.30


In [10]:
assert df_sales_prod_dataset.shape[1] == df_sales_block_dataset.shape[1], 'Products and blocks datasets shape mismatch'
max_date_number = df_sales_prod_dataset.shape[1] - 1

df_sales_prod_dataset.to_csv(os.path.join(data_out, f'revenue_products_day_0_day_{max_date_number}.csv'), sep=';')
df_sales_block_dataset.to_csv(os.path.join(data_out, f'revenue_blocks_day_0_day_{max_date_number}.csv'), sep=';')